In [4]:
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as st
sns.set()
import sys
from numpy.linalg import inv
# Provide the path to the sims emulator validation source code base.
sys.path.insert(0, '/Users/dananjayaliyanage/git/emulator-validation/src')
#sys.path.insert(0, '/Users/dananjayaliyanage/git/emulator-validation/emulator')
from configurations import load_design, transform_design
#from bayes_mcmc import *
from emulator import *
from calculations_load import trimmed_model_data,validation_data
from bins_and_cuts import *
import matplotlib.patches as mpatches
#from bayes_exp import Y_exp_data
from sklearn.preprocessing import StandardScaler

Loading Pb-Pb-2760 main calculations from model_calculations/production_500pts_Pb_Pb_2760/Obs/main.dat
WARNING! can not load model design calculations
Loading Pb-Pb-2760 MAP calculations from model_calculations/MAP/Grad/Obs/obs_Pb-Pb-2760.dat
No MAP calculations found for system Pb-Pb-2760
Loading Au-Au-200 main calculations from model_calculations/production_500pts_Au_Au_200/Obs/main.dat
WARNING! can not load model design calculations
Loading Au-Au-200 MAP calculations from model_calculations/MAP/Grad/Obs/obs_Au-Au-200.dat
No MAP calculations found for system Au-Au-200


In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

/Users/derek/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/derek/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/derek/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/derek/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework

In [6]:
# Put all model parametr names to an array

model_labels= ['norm','trento_p','sigma_k','nucleon_width',
               'dmin3','tau_R','alpha','shear_relax_time_factor','Tswitch']

viscous_eta=[r'$\eta \,$ at T={} GeV'.format(round(i,2)) 
             for i in np.linspace(0.135,0.4,10)]

viscous_bulk=[r'$\zeta \,$ at T={} GeV'.format(round(i,2))
              for i in np.linspace(0.135,0.4,10)]

all_model_labels=model_labels+viscous_eta+viscous_bulk

In [7]:
# Get all the observables list

nobs        =  0
observables =  []
obs_name    =  []

for obs, cent_list in obs_cent_list['Pb-Pb-2760'].items():
    observables.append(obs)
    n = np.array(cent_list).shape[0]
    for i in cent_list:
        obs_name.append(f'{obs}_{i}')
    #self._slices[obs] = slice(self.nobs, self.nobs + n)
    nobs += n

In [8]:
# Get the simulation results corresponding to design points
idf=0
Y   =  []
for ipt, data in enumerate(trimmed_model_data['Pb-Pb-2760']):
    row = np.array([])
    for obs in observables:
        values = np.array(data[idf][obs]['mean'])
        if np.isnan(values).sum() > 0:
            print("WARNING! FOUND NAN IN MODEL DATA WHILE BUILDING EMULATOR!")
            print("Design pt = " + str(pt) + "; Obs = " + obs)
        row = np.append(row, values)
    Y.append(row)
Y = np.array(Y)
print("Y_Obs shape[Ndesign, Nobs] = " + str(Y.shape))
#pca = PCA(copy=False, whiten=True, svd_solver='full')
#Z=pca.fit_transform(SS.fit_transform(Y))[:,:10]

KeyError: 'Pb-Pb-2760'

In [6]:
design, design_max, design_min, labels = prepare_emu_design('Pb-Pb-2760')
if len(delete_design_pts_set) > 0:
    print("Warning! Deleting "
          + str(len(delete_design_pts_set)) + " points from data")
    design = np.delete(design, list(delete_design_pts_set), 0)
alldesign=[design, design_max, design_min,labels]  

Loading main points from production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_main_PbPb-2760.dat
Loading main ranges from production_designs/500pts/design_pts_Pb_Pb_2760_production/design_ranges_main_PbPb-2760.dat
Summary of design : 
Note : Transforming design of viscosities
Warning! Deleting 15 points from data


In [7]:
# Get the simulation results for validaton corresponding to design points
YV=[]
nanrows=[]
for ipt, data in enumerate(validation_data['Pb-Pb-2760']):
    row = np.array([])
    for obs in observables:
        values = np.array(data[idf][obs]['mean'])
        if np.isnan(values).sum() > 0:
            print("WARNING! FOUND NAN IN MODEL DATA WHILE BUILDING EMULATOR!")
            print("Design pt = " + str(ipt) + "; Obs = " + obs)
            nanrows.append(ipt)
        row = np.append(row, values)
    YV.append(row)
YV = np.array(YV)
nanrows=np.array(nanrows)
print("Y_Val_Obs shape[Ndesign, Nobs] = " + str(YV.shape))
#pca = PCA(copy=False, whiten=True, svd_solver='full')
#Z=pca.fit_transform(SS.fit_transform(Y))[:,:10]

WARNING! FOUND NAN IN MODEL DATA WHILE BUILDING EMULATOR!
Design pt = 36; Obs = v42
WARNING! FOUND NAN IN MODEL DATA WHILE BUILDING EMULATOR!
Design pt = 43; Obs = v42
WARNING! FOUND NAN IN MODEL DATA WHILE BUILDING EMULATOR!
Design pt = 47; Obs = v42
WARNING! FOUND NAN IN MODEL DATA WHILE BUILDING EMULATOR!
Design pt = 73; Obs = v42
WARNING! FOUND NAN IN MODEL DATA WHILE BUILDING EMULATOR!
Design pt = 96; Obs = v42
Y_Val_Obs shape[Ndesign, Nobs] = (100, 123)


In [8]:
todeleterows=np.concatenate((nanrows,delete_design_pts_validation_set))

In [9]:
todeleterows

array([36, 43, 47, 73, 96, 10, 68, 93])

In [10]:
print(YV.shape)
YV_truncates=np.delete(YV,obj=todeleterows,axis=0)
print(f'after truccation shape{YV_truncates.shape}')

(100, 123)
after truccation shape(92, 123)


In [11]:
design_v,dsgn_max_v,dsgn_min_v,design_v_lbls=load_design(system_str='Pb-Pb-2760', pset='validation')
if len(todeleterows) > 0:
    print("Warning! Deleting " + str(len(todeleterows)) + " points from data")
    design_v= np.delete(np.array(design_v), obj=todeleterows, axis=0)
    print(f'Shape of validation design array {design_v.shape}')

Loading validation points from production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_validation_PbPb-2760.dat
Loading validation ranges from production_designs/500pts/design_pts_Pb_Pb_2760_production//design_ranges_validation_PbPb-2760.dat
Summary of design : 
Warning! Deleting 8 points from data
Shape of validation design array (92, 17)


In [12]:
design_v=transform_design(design_v)

In [5]:
# Model / data parametaers
#num_classes = 10
input_shape = (29,1)
#x_train= design.reshape(485,29,1)
#y_train= Y
#x_test=design_v.reshape(92,29,1)
#y_test=YV_truncates
# the data, split between train and test sets
#(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [14]:
# Scale images to the [0, 1] range
#x_train = x_train.astype("float32") / 255
#x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
#x_train = np.expand_dims(x_train, -1)
#x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (485, 29, 1)
485 train samples
92 test samples


In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv1D(100, kernel_size=3, activation="relu"),
        layers.MaxPooling1D(pool_size=3),
        layers.Conv1D(50, kernel_size=3, activation="relu"),
        layers.MaxPooling1D(pool_size=3),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(123, activation="softmax"),
    ]
)

model.summary()

TypeError: The added layer must be an instance of class Layer. Found: Tensor("input_1:0", shape=(?, 29, 1), dtype=float32)

In [16]:
batch_size = 50
epochs = 30

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 436 samples, validate on 49 samples
Epoch 1/30
436/436 [==============================] - 0s 566us/sample - loss: 250587.4243 - acc: 0.0528 - val_loss: 257169.6562 - val_acc: 0.0000e+00
Epoch 2/30
436/436 [==============================] - 0s 88us/sample - loss: 250581.7577 - acc: 0.1284 - val_loss: 257159.9531 - val_acc: 0.0000e+00
Epoch 3/30
436/436 [==============================] - 0s 70us/sample - loss: 250571.9258 - acc: 0.4495 - val_loss: 257147.2812 - val_acc: 1.0000
Epoch 4/30
436/436 [==============================] - 0s 66us/sample - loss: 250563.5461 - acc: 0.8922 - val_loss: 257145.4219 - val_acc: 1.0000
Epoch 5/30
436/436 [==============================] - 0s 73us/sample - loss: 250561.4269 - acc: 0.9885 - val_loss: 257145.4219 - val_acc: 1.0000
Epoch 6/30
436/436 [==============================] - 0s 62us/sample - loss: 250561.0664 - acc: 1.0000 - val_loss: 257145.4062 - val_acc: 1.0000
Epoch 7/30
436/436 [==============================] - 0s 65us/sample - loss:

In [17]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 245922.33763586957
Test accuracy: 1.0
